In [1]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
df = pd.read_csv('all_stocks_5yr.csv').fillna(0)
# df = df.iloc[:5000]
df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [3]:
df['target'] = 0

for idx, row in df.iterrows():
    
    today_close = df.iloc[idx]['close']
    try:
        tomorrow_high = df.iloc[idx+1]['high'] 
    except IndexError:
        break
    if (0.99*tomorrow_high) <= today_close:
        df.iat[idx, 7] = 0
    else:
        df.iat[idx, 7] = 1

In [4]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [5]:
df['year_encoded'] = LabelEncoder().fit_transform(df['year'])
df['month_encoded'] = LabelEncoder().fit_transform(df['month'])
df['day_encoded'] = LabelEncoder().fit_transform(df['day'])
df['name_encoded'] = LabelEncoder().fit_transform(df['Name'])


In [6]:
df['volume_scaled'] = StandardScaler().fit_transform(df[['volume']])
df['open_scaled'] = StandardScaler().fit_transform(df[['open']])
df['high_scaled'] = StandardScaler().fit_transform(df[['high']])
df['low_scaled'] = StandardScaler().fit_transform(df[['low']])
df['close_scaled'] = StandardScaler().fit_transform(df[['close']])

In [7]:
df.head()

,date,open,high,low,close,volume,Name,target,year,month,day,year_encoded,month_encoded,day_encoded,name_encoded,volume_scaled,open_scaled,high_scaled,low_scaled,close_scaled
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,1,2013,2,8,0,1,7,1,0.469964,-0.697812,-0.699105,-0.700725,-0.701242
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,0,2013,2,11,0,1,10,1,0.524544,-0.699661,-0.700225,-0.704559,-0.704220
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,1,2013,2,12,0,1,11,1,0.437583,-0.704179,-0.705317,-0.706217,-0.706171
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,1,2013,2,13,0,1,12,1,0.682994,-0.705719,-0.700938,-0.704663,-0.702166
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,1,2013,2,14,0,1,13,1,3.169926,-0.699147,-0.700735,-0.715957,-0.709046


In [8]:
split_date ='2017-05-01'
df_training = df.loc[df['date'] <= split_date]
df_test = df.loc[df['date'] > split_date]
pickle.dump((df_training, df_test), open("stock_data.pickle",'wb'))   
len(df_training), len(df_test)

(520897, 98143)

In [57]:
df_training, df_test = pickle.load(open("stock_data.pickle",'rb'))   
y_train = pd.DataFrame(df_training['target'].reset_index())
x_train = df_training.drop(['target', 'date', 'Name', 'high', 'close', 'open', 'low','volume', 'year', 'month', 'day'], 1).reset_index()
y_test = df_test['target'].reset_index()
x_test = pd.DataFrame(df_test.drop(['target', 'date', 'Name', 'high', 'close', 'open', 'low','volume', 'year', 'month', 'day'], 1).reset_index())

In [58]:
x_train.head()

,index,year_encoded,month_encoded,day_encoded,name_encoded,volume_scaled,open_scaled,high_scaled,low_scaled,close_scaled
0,0,0,1,7,1,0.469964,-0.697812,-0.699105,-0.700725,-0.701242
1,1,0,1,10,1,0.524544,-0.699661,-0.700225,-0.704559,-0.704220
2,2,0,1,11,1,0.437583,-0.704179,-0.705317,-0.706217,-0.706171
3,3,0,1,12,1,0.682994,-0.705719,-0.700938,-0.704663,-0.702166
4,4,0,1,13,1,3.169926,-0.699147,-0.700735,-0.715957,-0.709046


In [59]:
x_train.describe()

,index,year_encoded,month_encoded,day_encoded,name_encoded,volume_scaled,open_scaled,high_scaled,low_scaled,close_scaled
count,520897.000000,520897.000000,520897.000000,520897.000000,520897.000000,520897.000000,520897.000000,520897.000000,520897.000000,520897.000000
mean,309500.167308,1.746762,5.378344,14.734372,252.969449,0.006122,-0.037528,-0.037406,-0.037650,-0.037509
std,178752.783756,1.248651,3.427906,8.722888,145.945788,1.017422,0.918412,0.918690,0.918055,0.918349
min,0.000000,0.000000,0.000000,0.000000,0.000000,-0.497127,-0.852569,-0.853065,-0.852320,-0.836370
25%,154548.000000,1.000000,2.000000,7.000000,126.000000,-0.373724,-0.448784,-0.448513,-0.448827,-0.448752
50%,309841.000000,2.000000,5.000000,15.000000,255.000000,-0.255399,-0.229331,-0.229283,-0.229362,-0.229221
75%,464245.000000,3.000000,8.000000,22.000000,379.000000,0.000929,0.074843,0.074666,0.075175,0.074918
max,618844.000000,4.000000,11.000000,30.000000,504.000000,70.616963,18.160774,18.400398,18.316203,18.347698


In [60]:
y_train = y_train.values[:,1]
x_train = x_train.values[:,1:]
y_test = y_test.values[:,1]
x_test = x_test.values[:,1:]


In [61]:
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import classification_report
import xgboost

In [62]:
classifiers = [
    GaussianNB(),
    #  RidgeClassifier(tol=1e-2, solver="lsqr"),
    QuadraticDiscriminantAnalysis(),
    LinearDiscriminantAnalysis(),
    DecisionTreeClassifier(max_depth=5),
    KNeighborsClassifier(3, n_jobs=-1),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, n_jobs=-1),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    # SVC(kernel="linear", C=0.025, probability=True),
    # SVC(gamma=2, C=1, probability=True),
    # SVC(),
    MLPClassifier(alpha=1),
    xgboost.XGBClassifier()
    # GaussianProcessClassifier(1.0 * RBF(1.0), n_jobs=-1),
]

In [15]:
for clf in classifiers:
    print('_' * 80)
    print(clf.__class__.__name__)
    clf.fit(x_train, y_train)
    print('Train/test accuracy: ', clf.score(x_train, y_train), clf.score(x_test, y_test))
    print('Classification report of Test data')
    print(classification_report(y_test, clf.predict(x_test)))

________________________________________________________________________________
GaussianNB
Train/test accuracy:  0.6085905658892257 0.6479728559347075
Classification report of Test data
             precision    recall  f1-score   support

          0       0.66      0.97      0.78     63977
          1       0.45      0.05      0.09     34166

avg / total       0.58      0.65      0.54     98143

________________________________________________________________________________
QuadraticDiscriminantAnalysis
Train/test accuracy:  0.6158108032874061 0.6525274344578829
Classification report of Test data
             precision    recall  f1-score   support

          0       0.67      0.93      0.78     63977
          1       0.50      0.13      0.20     34166

avg / total       0.61      0.65      0.58     98143

________________________________________________________________________________
LinearDiscriminantAnalysis
Train/test accuracy:  0.6187998779029251 0.6573673109646129
Classific

/usr/lib/python3/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/lib/python3/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train/test accuracy:  0.6276020019312839 0.6532101117756742
Classification report of Test data
             precision    recall  f1-score   support

          0       0.66      0.98      0.79     63977
          1       0.53      0.04      0.07     34166

avg / total       0.61      0.65      0.54     98143



/usr/lib/python3/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [63]:
import keras
from keras.models import Sequential
from keras.models import Model 
from keras.layers import Input, Dense, Activation, Reshape, merge
from keras.layers import Concatenate, Dropout, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [64]:
model = Sequential([
        Dense(512, activation='relu', input_shape=(x_train.shape[1],)),
        Dropout(0.2),
        BatchNormalization(),
        Dense(512, activation='relu'),
        Dropout(0.2),
        BatchNormalization(),
        Dense(512, activation='relu'),
        Dropout(0.2),
        BatchNormalization(),
        Dense(1, activation='sigmoid')
    ])
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['acc'])


In [65]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=1, batch_size=128)

Train on 520897 samples, validate on 98143 samples
Epoch 1/1
520897/520897 [==============================] - 201s 386us/step - loss: 0.6773 - acc: 0.6016 - val_loss: 0.6456 - val_acc: 0.6523


In [76]:
input_year = Input(shape=(1,))
output_year = Embedding(5, 2, name='year_embedding')(input_year)
output_year = Reshape(target_shape=(2,))(output_year)

input_month = Input(shape=(1,))
output_month = Embedding(12, 3, name='month_embedding')(input_month)
output_month = Reshape(target_shape=(3,))(output_month)

input_day = Input(shape=(1,))
output_day = Embedding(31, 5, name='day_embedding')(input_day)
output_day = Reshape(target_shape=(5,))(output_day)

input_name = Input(shape=(1,))
output_name = Embedding(505, 10, name='name_embedding')(input_name)
output_name = Reshape(target_shape=(10,))(output_name)

input_volume = Input(shape=(1,))
output_volume = Dense(1)(input_volume)

input_open = Input(shape=(1,))
output_open = Dense(1)(input_open)

input_high = Input(shape=(1,))
output_high = Dense(1)(input_high)

input_low = Input(shape=(1,))
output_low = Dense(1)(input_low)

input_close = Input(shape=(1,))
output_close = Dense(1)(input_close)

input_model = [input_year, input_month, input_day, input_name, input_volume, 
               input_open, input_high, input_low, input_close]

output_embeddings = [output_year, output_month, output_day, output_name, output_volume, 
               output_open, output_high, output_low, output_close]

output_model = Concatenate()(output_embeddings)
output_model = Dense(1000, kernel_initializer="uniform")(output_model)
output_model = Activation('relu')(output_model)
output_model = Dense(500, kernel_initializer="uniform")(output_model)
output_model = Activation('relu')(output_model)
output_model = Dense(1)(output_model)
output_model = Activation('sigmoid')(output_model)

entity_model = Model(inputs=input_model, outputs=output_model)

entity_model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['acc'])


In [77]:
def split_features(X):
    X_list = [X[..., [i]] for i in range(9)]
    return X_list
entity_model.fit(split_features(x_train), y_train,   validation_data=(split_features(x_test), y_test), 
                 epochs=10, batch_size=128)

Train on 520897 samples, validate on 98143 samples
Epoch 1/10
520897/520897 [==============================] - 46s 89us/step - loss: 0.6308 - acc: 0.6459 - val_loss: 0.6812 - val_acc: 0.6432
Epoch 2/10
520897/520897 [==============================] - 44s 85us/step - loss: 0.5666 - acc: 0.7079 - val_loss: 0.7177 - val_acc: 0.6245
Epoch 3/10
520897/520897 [==============================] - 44s 85us/step - loss: 0.5260 - acc: 0.7380 - val_loss: 0.7635 - val_acc: 0.6371
Epoch 4/10
520897/520897 [==============================] - 45s 86us/step - loss: 0.5085 - acc: 0.7487 - val_loss: 0.7457 - val_acc: 0.6335
Epoch 5/10
520897/520897 [==============================] - 45s 86us/step - loss: 0.4967 - acc: 0.7568 - val_loss: 0.7561 - val_acc: 0.6269
Epoch 6/10
520897/520897 [==============================] - 45s 86us/step - loss: 0.4875 - acc: 0.7626 - val_loss: 0.7997 - val_acc: 0.6375
Epoch 7/10
520897/520897 [==============================] - 45s 87us/step - loss: 0.4798 - acc: 0.7671 - val_

In [78]:

inputs = Input(shape=(x_train.shape[1],))

# ATTENTION PART STARTS HERE
attention_probs = Dense(x_train.shape[1], activation='softmax', name='attention_vec')(inputs)
attention_mul = Concatenate()([inputs, attention_probs])
# ATTENTION PART FINISHES HERE

attention_mul = Dense(128)(attention_mul)
attention_mul = Dense(128)(attention_mul)
output = Dense(1, activation='sigmoid')(attention_mul)
attention_model = Model(input=[inputs], output=output)
attention_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['acc'])

attention_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

/home/raja/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  if sys.path[0] == '':


Train on 520897 samples, validate on 98143 samples
Epoch 1/10
520897/520897 [==============================] - 107s 205us/step - loss: 0.8949 - acc: 0.5858 - val_loss: 0.6475 - val_acc: 0.6520
Epoch 2/10
520897/520897 [==============================] - 106s 203us/step - loss: 0.6681 - acc: 0.6105 - val_loss: 0.6544 - val_acc: 0.6519
Epoch 3/10
520897/520897 [==============================] - 104s 200us/step - loss: 0.6673 - acc: 0.6110 - val_loss: 0.6472 - val_acc: 0.6523
Epoch 4/10
520897/520897 [==============================] - 105s 202us/step - loss: 0.6669 - acc: 0.6112 - val_loss: 0.6472 - val_acc: 0.6523
Epoch 5/10
520897/520897 [==============================] - 105s 201us/step - loss: 0.6662 - acc: 0.6109 - val_loss: 0.6538 - val_acc: 0.6394
Epoch 6/10
520897/520897 [==============================] - 105s 201us/step - loss: 0.6659 - acc: 0.6110 - val_loss: 0.6443 - val_acc: 0.6521
Epoch 7/10
520897/520897 [==============================] - 105s 202us/step - loss: 0.6657 - acc: